In [ ]:
!pip install azure

In [1]:
from azure.storage.table import TableService, Entity, TableBatch
from collections import defaultdict
import uuid, datetime, random
import pandas as pd

In [2]:
table_service = TableService(account_name='playland', account_key='<copied from portal>')

In [5]:
table_service.create_table('simulatedData')
#table_service.delete_table('simulatedData')

True

In [6]:
for p in range(1,10):
    batch = TableBatch()
    temp = random.randint(65,100)
    hum = random.uniform(0.3, 0.8)
    press = 101000
    for i in range(1,1001):
        e = {
            'PartitionKey': 'device' + str(p),
            'RowKey': str(uuid.uuid4()),
            'timestamp': str(datetime.datetime.now()),
            'temperature': random.randint(temp-5, temp+5),
            'humidity': random.uniform(hum-0.1,hum+0.1),
            'pressure': random.randint(press-1000,press+1000)
        }
        batch.insert_entity(e)
        
        if (i % 25 == 0):
            print("adding", i, "records for device", p)
            table_service.commit_batch('simulatedData', batch)
            batch = TableBatch()


adding 25 records for device 1
adding 50 records for device 1
adding 75 records for device 1
adding 100 records for device 1
adding 125 records for device 1
adding 150 records for device 1
adding 175 records for device 1
adding 200 records for device 1
adding 225 records for device 1
adding 250 records for device 1
adding 275 records for device 1
adding 300 records for device 1
adding 325 records for device 1
adding 350 records for device 1
adding 375 records for device 1
adding 400 records for device 1
adding 425 records for device 1
adding 450 records for device 1
adding 475 records for device 1
adding 500 records for device 1
adding 525 records for device 1
adding 550 records for device 1
adding 575 records for device 1
adding 600 records for device 1
adding 625 records for device 1
adding 650 records for device 1
adding 675 records for device 1
adding 700 records for device 1
adding 725 records for device 1
adding 750 records for device 1
adding 775 records for device 1
adding 800 

adding 450 records for device 7
adding 475 records for device 7
adding 500 records for device 7
adding 525 records for device 7
adding 550 records for device 7
adding 575 records for device 7
adding 600 records for device 7
adding 625 records for device 7
adding 650 records for device 7
adding 675 records for device 7
adding 700 records for device 7
adding 725 records for device 7
adding 750 records for device 7
adding 775 records for device 7
adding 800 records for device 7
adding 825 records for device 7
adding 850 records for device 7
adding 875 records for device 7
adding 900 records for device 7
adding 925 records for device 7
adding 950 records for device 7
adding 975 records for device 7
adding 1000 records for device 7
adding 25 records for device 8
adding 50 records for device 8
adding 75 records for device 8
adding 100 records for device 8
adding 125 records for device 8
adding 150 records for device 8
adding 175 records for device 8
adding 200 records for device 8
adding 225

In [7]:
parts = defaultdict(int)
es = table_service.query_entities('simulatedData')
for e in es:
    parts[e.PartitionKey] += 1

print(parts)

defaultdict(<class 'int'>, {'device6': 1000, 'device7': 1000, 'device9': 1000, 'device4': 1000, 'device5': 1000, 'device1': 1000, 'device3': 1000, 'device8': 1000, 'device2': 1000})


In [3]:
class TableReader(object):
    def __init__(self, tableService, tableName, columns):
        self.tableService = tableService
        self.tableName = tableName
        self.columns = columns
        self.gen = iter(tableService.query_entities(tableName, select=','.join(columns)))
        self.row = next(self.gen)
        self.header = '","'.join([x for x in columns]).join(('"', '"\n'))
        
    def read(self, n=0):
        result = ''
        try:
            if len(self.header) > 0:
                result = self.header
                self.header = ''
            else:
                result = '","'.join([str(self.row[x]) for x in self.columns]).join(('"', '"\n'))
                self.row = next(self.gen)
        except StopIteration:
            pass
        return result

In [5]:
tr = TableReader(table_service, 
                 'simulatedData', 
                 ['RowKey',
                  'PartitionKey',
                  'temperature',
                  'humidity',
                  'pressure'])
df = pd.read_csv(tr, nrows=400)
df

,RowKey,PartitionKey,temperature,humidity,pressure
0,006e093f-0ded-49d4-a726-849cab9d3e70,device1,65,0.689191,100321
1,0072cafa-e6ae-407a-b625-bacd7a51f8a1,device1,70,0.613485,100707
2,00be37ba-6d0e-48e5-ae60-e1d0d77bea7b,device1,65,0.631592,100990
3,00d9f646-477c-4f89-abea-9b9e88d09876,device1,62,0.594895,101368
4,00eecb3f-92ab-4363-bd5a-b241799b0720,device1,61,0.654789,100638
5,01116c6a-8885-4cef-8f14-0ebf41bf3d4e,device1,62,0.675961,101001
6,0130c872-5fc8-421f-9390-54b5160a0cc1,device1,66,0.655808,101698
7,02102f73-95af-4437-b0b6-c7dfdf3e3e50,device1,66,0.613902,100320
8,0231c8ae-a782-4fc4-ab36-ac0dd8dfa2b1,device1,70,0.580904,101658
9,0255481f-bc02-4c98-929e-7723159737ed,device1,64,0.723938,100523


In [6]:
tr = TableReader(table_service, 
                 'simulatedData', 
                 ['RowKey',
                  'PartitionKey',
                  'temperature',
                  'humidity',
                  'pressure'])
df = pd.read_csv(tr, skiprows=400, nrows=400)
df

,644a34c4-d482-42b3-8374-bc9a1d6811ce,device1,63,0.7305901902728043,100899
0,645765db-b129-454c-9617-6857e3721160,device1,62,0.746594,100014
1,645955fa-3afb-4c78-911c-8d938b4ffd07,device1,68,0.742855,100621
2,648c40c8-79b8-4315-b821-0e05054455e7,device1,69,0.610468,100420
3,649ff78c-796e-4b88-af74-ae3110c4c1ae,device1,63,0.666317,101563
4,64a9e989-e321-42ff-9119-b29deab6614f,device1,69,0.664429,101881
5,64e05a48-b34e-41fb-b6f5-0262421a147f,device1,71,0.674600,101736
6,64e8acfb-6dd6-4bf6-bfa3-1976f313ac60,device1,68,0.749048,100161
7,65098e00-2fce-42a5-abfb-a953c73d6927,device1,66,0.617726,100539
8,651fe0bc-7a77-4332-b23e-11dcefcb5de8,device1,64,0.640836,100969
9,65e3e702-7a55-4c2e-800e-e8e569b6d1c9,device1,64,0.645157,100272
